In [1]:
import os
# import json
import math
import torch
from itertools import product
from dataclasses import dataclass, field

from gamesopt.attacks import Attack
from gamesopt.optimizer import Optimizer
from gamesopt.aggregator import Aggregator
from gamesopt.games import Game, create_matrix, create_bias
from gamesopt.train_distributed import train

# %matplotlib widget
%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd()) + '-lr'

In [3]:
# class Config(BaseConfig):
@dataclass
class Config():
    n_iter: int = 50001

    n_peers: int = 20
    n_byzan: int = 4

    # game: Game = Game.Quadratic
    game: Game = field(default_factory=lambda: Game.Quadratic)
    num_samples: int = 1000
    dim: int = 25
    with_bias: bool = True
    mu: float = 1e-1
    ell: float = 1e3

    optimizer: Optimizer = None
    batch_size: int = None
    lr: float = None
    alpha: float = None
    sigmaC: float = None

    attack: Attack = None
    n_attacking: int = None
    attack_param: float = None

    use_bucketing: bool = None
    bucketing_s: int = 2

    aggregator: Aggregator = None
    aggregator_param_a: int = 10  # trimmed_mean_b=10; krum_m=2; ,clipping_n_iter=3 clipping_tau=10
    aggregator_param_b: float = 0.1 # rfa_T=10,rfa_nu=0.1    , 0.1)  # rfa_T=10,rfa_nu=0.1

In [4]:
@dataclass
class Data():
    def __init__(self, config):
        self.matrix = create_matrix(config.dim, config.num_samples,
                                    config.mu, config.ell,
                                    with_bias=config.with_bias)
        self.bias = create_bias(config.dim, config.num_samples,
                                with_bias=config.with_bias)
        self.true = torch.linalg.solve(self.matrix.mean(dim=0),
                                       -self.bias.mean(dim=0))
        self.players = self.true + .1/math.sqrt(2 * config.dim)

In [5]:
base_config = Config()
data = Data(base_config)
# base_config_items = set(base_config.__dict__.items())

### ALIE and IPM

In [4]:
1/2/1000/30

1.6666666666666667e-05

In [6]:
ell = base_config.ell
for lr, at, ap, bs in product([1/2/ell/100, 1/2/ell/50, 1/2/ell/25],
                          [Attack.ALIE],
                          [1e1],
                          [10]):

    # SGDA
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.optimizer = Optimizer.SGDA
    config.aggregator = Aggregator.Mean
    config.n_attacking = config.n_byzan
    # config_items = set(config.__dict__.items())
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'SGDA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    # json.dumps(dict(sorted(config_items - base_config_items)))
    train(config, data)

    # SGDARA
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.optimizer = Optimizer.SGDARA
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'SGDARA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # MSGDARA
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.optimizer = Optimizer.MSGDARA
    config.alpha = 0.1
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA M'
    os.environ['MLFLOW_RUN_NAME'] = 'MSGDARA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # SEGRA
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = int(config.n_iter / 2) + 1
    config.optimizer = Optimizer.SEGRA
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'SEGRA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # RDEG
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = int(config.n_iter / 2) + 1
    config.optimizer = Optimizer.RDEG
    config.aggregator = Aggregator.UnivariateTM
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'RDEG'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # SGDACC
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = config.n_iter * 2 - 1
    config.optimizer = Optimizer.SGDACC
    config.aggregator = Aggregator.Mean
    config.sigmaC = 100.
    config.n_attacking = 1
    os.environ['MLFLOW_RUN_TAG'] = 'CC'
    os.environ['MLFLOW_RUN_NAME'] = 'SGDACC'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # SEGCC
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = config.n_iter + 1
    config.optimizer = Optimizer.SEGCC
    config.aggregator = Aggregator.Mean
    config.sigmaC = 100.
    config.n_attacking = 1
    os.environ['MLFLOW_RUN_TAG'] = 'CC'
    os.environ['MLFLOW_RUN_NAME'] = 'SEGCC'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)


Trying port 44793
Trying port 34133
Trying port 26294
Trying port 25798
Trying port 30450
Trying port 7887
Trying port 22007
Trying port 35135
Trying port 41626
Trying port 27818
Trying port 21136
Trying port 20490
Trying port 29712
Trying port 47031
Trying port 24401
Trying port 18678
Trying port 14004
Trying port 23237
Trying port 29744
Trying port 9561
Trying port 2846


### RN

In [9]:
ell = base_config.ell
for lr, at, ap, bs in product([1/2/ell/100, 1/2/ell/50, 1/2/ell/25],
                          [Attack.RN],
                          [1e3],
                          [1]):

    # # SGDA
    # config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    # config.optimizer = Optimizer.SGDA
    # config.aggregator = Aggregator.Mean
    # config.n_attacking = config.n_byzan
    # # config_items = set(config.__dict__.items())
    # os.environ['MLFLOW_RUN_TAG'] = 'RA'
    # os.environ['MLFLOW_RUN_NAME'] = 'SGDA'
    # os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    # # json.dumps(dict(sorted(config_items - base_config_items)))
    # train(config, data)

    # SGDARA
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.optimizer = Optimizer.SGDARA
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'SGDARA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # MSGDARA
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.optimizer = Optimizer.MSGDARA
    config.alpha = 0.1
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA M'
    os.environ['MLFLOW_RUN_NAME'] = 'MSGDARA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # SEGRA
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = int(config.n_iter / 2) + 1
    config.optimizer = Optimizer.SEGRA
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'SEGRA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # RDEG
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = int(config.n_iter / 2) + 1
    config.optimizer = Optimizer.RDEG
    config.aggregator = Aggregator.UnivariateTM
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'RDEG'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # SGDACC
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = config.n_iter * 2 - 1
    config.optimizer = Optimizer.SGDACC
    config.aggregator = Aggregator.Mean
    config.sigmaC = 100.
    config.n_attacking = 1
    os.environ['MLFLOW_RUN_TAG'] = 'CC'
    os.environ['MLFLOW_RUN_NAME'] = 'SGDACC'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)

    # SEGCC
    config = Config(lr=lr, attack=at, attack_param=ap, batch_size=bs)
    config.n_iter = config.n_iter + 1
    config.optimizer = Optimizer.SEGCC
    config.aggregator = Aggregator.Mean
    config.sigmaC = 100.
    config.n_attacking = 1
    os.environ['MLFLOW_RUN_TAG'] = 'CC'
    os.environ['MLFLOW_RUN_NAME'] = 'SEGCC'
    os.environ['MLFLOW_RUN_TITLE'] = '%s (%.e), bs=%i' % (at, ap, bs)
    train(config, data)


Trying port 39549
Trying port 33917
Trying port 11306
Trying port 10673
Trying port 48357
Trying port 32734
Trying port 22625
Trying port 18747
Trying port 41572
Trying port 32289
Trying port 13871
Trying port 28296
Trying port 28397
Trying port 6762
Trying port 17966
Trying port 3775
Trying port 9862
Trying port 1359


In [ ]:
ell = base_config.ell
for lr, at, bs in product([1/2/ell/100, 1/2/ell/50, 1/2/ell/25],
                          [Attack.BF],
                          [10]):

    # # SGDA
    # config = Config(attack=at, batch_size=bs)
    # config.optimizer = Optimizer.SGDA
    # config.aggregator = Aggregator.Mean
    # config.n_attacking = config.n_byzan
    # # config_items = set(config.__dict__.items())
    # os.environ['MLFLOW_RUN_TAG'] = 'RA'
    # os.environ['MLFLOW_RUN_NAME'] = 'SGDA'
    # os.environ['MLFLOW_RUN_TITLE'] = '%s, bs=%i' % (at, bs)
    # # json.dumps(dict(sorted(config_items - base_config_items)))
    # train(config, data)

    # SGDARA
    config = Config(lr=lr, attack=at, batch_size=bs)
    config.optimizer = Optimizer.SGDARA
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'SGDARA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s, bs=%i' % (at, bs)
    train(config, data)

    # MSGDARA
    config = Config(lr=lr, attack=at, batch_size=bs)
    config.optimizer = Optimizer.MSGDARA
    config.alpha = 0.1
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA M'
    os.environ['MLFLOW_RUN_NAME'] = 'MSGDARA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s, bs=%i' % (at, bs)
    train(config, data)

    # SEGRA
    config = Config(lr=lr, attack=at, batch_size=bs)
    config.n_iter = int(config.n_iter / 2) + 1
    config.optimizer = Optimizer.SEGRA
    config.aggregator = Aggregator.RFA
    config.use_bucketing = True
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'SEGRA'
    os.environ['MLFLOW_RUN_TITLE'] = '%s, bs=%i' % (at, bs)
    train(config, data)

    # RDEG
    config = Config(lr=lr, attack=at, batch_size=bs)
    config.n_iter = int(config.n_iter / 2) + 1
    config.optimizer = Optimizer.RDEG
    config.aggregator = Aggregator.UnivariateTM
    config.n_attacking = config.n_byzan
    os.environ['MLFLOW_RUN_TAG'] = 'RA'
    os.environ['MLFLOW_RUN_NAME'] = 'RDEG'
    os.environ['MLFLOW_RUN_TITLE'] = '%s, bs=%i' % (at, bs)
    train(config, data)

    # SGDACC
    config = Config(lr=lr, attack=at, batch_size=bs)
    config.n_iter = config.n_iter * 2 - 1
    config.optimizer = Optimizer.SGDACC
    config.aggregator = Aggregator.Mean
    config.sigmaC = 100.
    config.n_attacking = 1
    os.environ['MLFLOW_RUN_TAG'] = 'CC'
    os.environ['MLFLOW_RUN_NAME'] = 'SGDACC'
    os.environ['MLFLOW_RUN_TITLE'] = '%s, bs=%i' % (at, bs)
    train(config, data)

    # SEGCC
    config = Config(lr=lr, attack=at, batch_size=bs)
    config.n_iter = config.n_iter + 1
    config.optimizer = Optimizer.SEGCC
    config.aggregator = Aggregator.Mean
    config.sigmaC = 100.
    config.n_attacking = 1
    os.environ['MLFLOW_RUN_TAG'] = 'CC'
    os.environ['MLFLOW_RUN_NAME'] = 'SEGCC'
    os.environ['MLFLOW_RUN_TITLE'] = '%s, bs=%i' % (at, bs)
    train(config, data)


Trying port 1352
Trying port 30453
Trying port 19011
Trying port 6779
Trying port 30985
Trying port 37144
Trying port 28314
Trying port 6545
Trying port 46130
Trying port 16076
Trying port 27979
Trying port 21000
Trying port 25642
Trying port 28953
Trying port 18129
Trying port 30293
